# Dimensional Analysis - The Quantity Class

Physical equations usually involve variables with both a value and units - aka their "concrete value" and "dimension". Dimensional analysis involves making sure that the units/dimensions of the variables are commensurate when combined. That is when two varibles are added, subtracted or equated they have the same dimension in terms of their base SI units.

Dimensional analysis by-hand is often a pain, especially when variables are raised by exponents and expressed in different, often non-base, units. To prevent headaches I wrote the `Quantity` class to represent a concrete value and its units.


#### References

https://en.wikipedia.org/wiki/Dimensional_analysis

https://en.wikipedia.org/wiki/SI_derived_unit


#### Similar packages

SymPy https://docs.sympy.org/latest/modules/physics/units/index.html

SymDim

Pint

In [6]:
from quantity import Quantity

import numpy as np

## Specifying a quantity

A `Quantity` is defined in terms of a concrete value and a variable number of unit arguments. Each unit argument consists of up to three parts:

$$ value \ (unit \times  10^n)^{p} $$ 
<center> Quantity(value, (n, unit, p)) </center>

i) An base-10 multiplier (n). This can be given as an integer (-6 for micro, -2 for centi, etc.) or short string ('mu' for micro-, 'c' for centi-, etc.). Allowable values are given n the table below.

ii) The unit name ('g' for gram, 'm' for metre, etc.). Any value is allowable.

iii) An exponent (p) - the dimension of the unit.

| prefix | base-10 multipler | string arg |
|---|---|---|
|atto|-18|a|
|femto|-15|f|
|pico|-12|p|
|nano|-9|n|
|micro|-6|mu|
|milli|-3|m|
|centi|-2|c|
|deci|-1|d|
|deca|1|da|
|hecto|2|2|
|kilo|3|k|
|mega|6|M|
|giga|9|G|
|tera|12|T|
|peta|15|15|


The argument for the unit can exclude the base-10 multiplier and exponent if these are superfluous. For instance, the following declarations are equivalent for 3.5 grams.

In [2]:
mass = Quantity(3.5, (0, "g", 1))
print(mass)

mass = Quantity(3.5, (0, "g"))
print(mass)

mass = Quantity(3.5, ("g", 1))
print(mass)

mass = Quantity(3.5, "g")
print(mass)

3.5 g^1
3.5 g^1
3.5 g^1
3.5 g^1


Quantities with compound units are declared with multiple unit arguments.

In [3]:
acceleration_of_freefall = Quantity(9.8, "m", ("s", 2))
print(acceleration_of_freefall)

resistance = Quantity(2.1, ("k" "g"), ("m", 2), ("s", -3), ("A", 2))
print(resistance)

9.8 m^1.s^2
2.1 kg^1.m^2.s^-3.A^2


The concrete value can be a numpy array.

In [7]:
speeds = Quantity(np.array([0.0, 0.5, 1.5, 5.6]), "m", ("s", -1))
print(speeds)

[0.  0.5 1.5 5.6] m^1.s^-1


Base SI units are specified by the following strings.

|SI unit|unit string|
|--|--|
|kilogram|kg|
|meter|m|
|second|s|
|Ampere|A|
|Kelvin|K|

A number of derived (non-SI) units are also recognised.

|Category|derived unit|unit string|
|--|--|--|
|**Mechanical**|Joule|J|
||Newton|N|
||Watt|W|
||Pascal|Pa|
|**Electrical**|Ohm|Ohm|
||Siemen|S|
||Volt|V|
||Farad|F|
||Coulomb|C|
|**Non-compound**|Litre|L|
||Hertz|Hz|
||gram|g|


They can be converted in SI units by calling `to_base_units()`.

In [4]:
pressure = Quantity(1.0, "Pa")
print(pressure.to_base_units())

resistance = Quantity(1.0, "Ohm")
print(resistance.to_base_units())

volume = Quantity(2.0, "L")
print(volume.to_base_units())

1.0 kg^1.m^-1.s^-2
1.0 kg^1.m^2.s^-3.A^-2
0.002 m^3


## Arithmetic

In [6]:
mass = Quantity(2.0, (-3, "kg")) + Quantity(3.0, "g")
print(mass)

mass = Quantity(2.0, (-3, "kg")) - Quantity(3.0, "g")
print(mass)

mass = Quantity(2.0, (-3, "kg")) * Quantity(3.0, "g")
print(mass)

mass = Quantity(2.0, (-3, "kg")) / Quantity(3.0, "g")
print(mass)

0.005 kg^1
-0.001 kg^1
6e-06 kg^2
0.6666666666666666 


## Commensurability